In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import stock

# 评估买入点和卖出点精度

In [2]:
"""对买入点和卖出进行评估"""
def EvaluationBuySell(k,buypt,sellpt):
    pass

# 买入点: kdj J斜率向上J<25或者上一个J<25 时的下一个kdj金叉点,没判断出来的使用MACD金叉和死叉点

In [3]:
"""
买入点分析
k kline
m kdj J斜率
J kdj J
kdd kdj K-D
bi,ei
macd MACD指标
"""
def kdjBuyJ20Pts(k):
    bi = 0
    ei = len(k)
    kdJ = stock.kdj(k,9)
    J = kdJ[:,2]
    m = stock.SlopeRates(kdJ[:,2])
    kdd = kdJ[:,0]-kdJ[:,1]
    macd,_ = stock.macd(k)

    prev = m[bi]
    buy = -1
    buyi = 0
    selli = 0
    sell = -1
    buytr = []
    selltr = []
    prevkdd = kdd[0]
    for i in range(bi+1,ei):
        macD = m[i]
        if prev<0 and macD>0 and (J[i-1]<25 or J[i]<25):
            buy = k[i,4]
            buyi = i
        if prevkdd < 0 and kdd[i]>0 and (macd[i]<0 or (macd[i-1]<0 and macd[i]>0)) and i-buyi<=5:
            buytr.append([i])
            
        if prev>0 and macD<0 and (J[i-1]>80 or J[i]>80):
            sell = k[i,4]
            selli = i
        if prevkdd > 0 and kdd[i]<0 and (macd[i]>0 or (macd[i-1]>0 and macd[i]<0)) and i-selli<=5:
            selltr.append([i])   
            
        prevkdd = kdd[i]
        prev = macD
    return np.array(buytr),np.array(selltr)

In [4]:
c,kline,kdate = stock.loadKline('SH601555')
kdJ = stock.kdj(kline,9)
macD,emA = stock.macd(kline)

In [5]:
buys,sells = kdjBuyJ20Pts(kline)
minpts,maxpts = stock.MacdBestPt(kline,macD)
print("buy:",len(buys),len(minpts),"sell:",len(sells),len(maxpts))
stock.Kline(kline,len(kline)-100,len(kline),vlines={"green":minpts,"cyan":buys,"red":maxpts,"darkorange":sells},kdate=kdate,boll=True,volume=True,macd=macD,kdj=kdJ)

buy: 106 68 sell: 93 67


Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle())),…

Output()

# 买入点: J斜率向上J<25或者上一个J<25 时的下一个kdj金叉点,没判断出来的使用MACD金叉和死叉点  （加入均线MA限制，买点必须小于MA20,卖点大于MA20）

In [6]:
"""
买入点分析
k kline
m kdj J斜率
J kdj J
kdd kdj K-D
bi,ei
macd MACD指标
"""
def kdjMa20Pts(k):
    bi = 0
    ei = len(k)
    kdJ = stock.kdj(k,9)
    J = kdJ[:,2]
    m = stock.SlopeRates(kdJ[:,2])
    kdd = kdJ[:,0]-kdJ[:,1]
    macd,_ = stock.macd(k)
    ma20 = stock.ma(k,20)
    
    prev = m[bi]
    buy = -1
    buyi = 0
    selli = 0
    sell = -1
    buytr = []
    selltr = []
    prevkdd = kdd[0]
    for i in range(bi+1,ei):
        macD = m[i]
        if prev<0 and macD>0 and (J[i-1]<25 or J[i]<25):
            buy = k[i,4]
            buyi = i
        if prevkdd < 0 and kdd[i]>0 and (macd[i]<0 or (macd[i-1]<0 and macd[i]>0)) and i-buyi<=5 and (ma20[i]>k[i,3] or ma20[i-1]>k[i-1,4]):
            buytr.append([i])
            
        if prev>0 and macD<0 and (J[i-1]>80 or J[i]>80):
            sell = k[i,4]
            selli = i
        if prevkdd > 0 and kdd[i]<0 and (macd[i]>0 or (macd[i-1]>0 and macd[i]<0)) and i-selli<=5 and (ma20[i]<k[i,3] or ma20[i-1]<k[i-1,4]):
            selltr.append([i])   
            
        prevkdd = kdd[i]
        prev = macD
    return np.array(buytr),np.array(selltr)

In [7]:
buys,sells = kdjMa20Pts(kline)
minpts,maxpts = stock.MacdBestPt(kline,macD)
print("buy:",len(buys),len(minpts),"sell:",len(sells),len(maxpts))
stock.Kline(kline,len(kline)-100,len(kline),vlines={"green":minpts,"cyan":buys,"red":maxpts,"darkorange":sells},kdate=kdate,boll=20,volume=True,macd=macD,kdj=kdJ)

buy: 96 68 sell: 79 67


Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle())),…

Output()